In [3]:
import pandas as pd
import folium
from folium.plugins import FloatImage, TimestampedGeoJson

# Read the CSV file into a Pandas DataFrame
data = pd.read_csv('./Data.csv')

# Filter the DataFrame for the year 2015
data_2015 = data[data['YEAR'] == 2015]

# Create a base map
m = folium.Map(location=[data_2015['LATITUDE'].mean(), data_2015['LONGITUDE'].mean()], zoom_start=5)

# Define a function to format the marker color based on SEVERITY_CODE
def get_marker_color(severity_code):
    color_map = {
        0: 'green',
        1: 'yellow',
        2: 'orange',
        3: 'red'
    }
    return color_map.get(severity_code, 'gray')

# Create GeoJSON data for the year 2015
features = []
for index, row in data_2015.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['LONGITUDE'], row['LATITUDE']]
        },
        "properties": {
            "time": row['datetime'],
            "popup": f"Severity: {row['SEVERITY_CODE']}",
            "icon": "circle",
            "iconstyle": {
                "fillColor": get_marker_color(row['SEVERITY_CODE']),
                "fillOpacity": 0.8,
                "radius": 6,
                "color": "black",
                "weight": 1
            }
        }
    }
    features.append(feature)

#Creates geojson_data object to create a FeatureCollection
geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

# Create the TimestampedGeoJson layer
TimestampedGeoJson(
    geojson_data,
    period="P1M", 
    duration="P1Y",
    add_last_point=True
).add_to(m)

# Save the interactive map to an HTML file
m.save('./interactive_map_coral.html')